In [121]:
import xlwings as xw
import pandas as pd

gst_file = r"C:\Users\s0656\Downloads\PURCHASE LEDGER.xlsx"
tally_file = r"C:\Users\s0656\Downloads\TALLY.xlsx"

gst_sheet_name = "Sheet1"
tally_sheet_name = "Dlo"

gst_wb = xw.Book(gst_file)
tally_wb = xw.Book(tally_file)

gst_sheet = gst_wb.sheets[gst_sheet_name]
tally_sheet = tally_wb.sheets[tally_sheet_name]

# Load the data from the Excel sheets into DataFrames
gst_data = pd.DataFrame(gst_sheet.range("A6").options(pd.DataFrame, index=False, expand='table').value)
tally_data = pd.DataFrame(tally_sheet.range("A7:H1589").options(pd.DataFrame, index=False).value)
# tally_data = pd.DataFrame(tally_sheet.range("A7:H1539").value)

In [122]:
gst_data.columns


Index(['Sl.No', 'Date', 'Draw No.', 'Total Tickets', 'TCS', 'Net Paid', 'Net'], dtype='object')

In [123]:
gst_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1287 entries, 0 to 1286
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Sl.No          1287 non-null   object        
 1   Date           1285 non-null   datetime64[ns]
 2   Draw No.       1285 non-null   object        
 3   Total Tickets  1286 non-null   float64       
 4   TCS            1286 non-null   float64       
 5   Net Paid       1286 non-null   object        
 6   Net            1286 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 70.5+ KB


In [124]:
gst_data

,Sl.No,Date,Draw No.,Total Tickets,TCS,Net Paid,Net
0,1.0,2023-04-01,W- 713,3300.0,0.0,106125.\n00,106125.0
1,2.0,2023-04-01,SS- 359,3600.0,0.0,116157.\n00,116157.0
2,3.0,2023-04-01,KN- 464,600.0,0.0,19500.0\n0,19500.0
3,4.0,2023-04-01,FF- 44,600.0,0.0,24843.0\n0,24843.0
4,5.0,2023-04-01,AK- 593,600.0,0.0,19360.0\n0,19360.0
...,...,...,...,...,...,...,...
1282,1283.0,2024-03-29,SS- 409,600.0,24.0,19524.0\n0,19524.0
1283,1284.0,2024-03-30,AK- 645,600.0,24.0,19384.0\n0,19384.0
1284,1285.0,2024-03-30,SS- 409,1200.0,48.0,38486.0\n0,38486.0
1285,Total,NaT,None,1637585.0,64493.0,56188224\n4,56188224.0


In [125]:
tally_data.columns

Index(['Date', 'Particulars', 'Voucher Type', 'Narration', 'Quantity', 'Value',
       'Gross Total', 'Purchase of Lottery'],
      dtype='object')

In [126]:
tally_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1582 entries, 0 to 1581
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 308 non-null    datetime64[ns]
 1   Particulars          1582 non-null   object        
 2   Voucher Type         308 non-null    object        
 3   Narration            0 non-null      object        
 4   Quantity             1582 non-null   float64       
 5   Value                1582 non-null   float64       
 6   Gross Total          309 non-null    float64       
 7   Purchase of Lottery  309 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(3)
memory usage: 99.0+ KB


In [127]:
tally_data['Date'] = tally_data['Date'].ffill()
tally_data

,Date,Particulars,Voucher Type,Narration,Quantity,Value,Gross Total,Purchase of Lottery
0,2023-04-01,Dlo,Purchase,None,8700.0,285985.0,285985.0,285985.0
1,2023-04-01,WW,None,None,3300.0,106125.0,NaN,NaN
2,2023-04-01,SS,None,None,3600.0,116157.0,NaN,NaN
3,2023-04-01,KN,None,None,600.0,19500.0,NaN,NaN
4,2023-04-01,FF,None,None,600.0,24843.0,NaN,NaN
...,...,...,...,...,...,...,...,...
1577,2024-03-29,SS,None,None,600.0,19524.0,NaN,NaN
1578,2024-03-30,Dlo,Purchase,None,1800.0,57870.0,57870.0,57870.0
1579,2024-03-30,AK,None,None,600.0,19386.0,NaN,NaN
1580,2024-03-30,SS,None,None,1200.0,38484.0,NaN,NaN


In [128]:
tally_data['checked'] = False
gst_data['checked'] = False
tally_data['correct'] = False
gst_data['correct'] = False
tally_data['remark'] = None

In [129]:
import re

def match_name(tname, name):
    if tname in ("AK", "SS", "FF", "KN", "NR", "KR"):
        pattern = rf'^{tname}-\s*\d+$'
        return bool(re.match(pattern, name))
    if tname == "WW":
        pattern = rf'^W-\s*\d+$'
        return bool(re.match(pattern, name))
    if tname == "Vishu" and name == "BR-91":
        return True
    elif tname == "Mb" and name == "BR-92":
        return True
    elif tname == "OB" and name == "BR-93":
        return True
    elif tname == "PB" and name == "BR-94":
        return True
    elif tname == "X Mas BR" and name == "BR-95":
        return True
    elif tname == "Summer BR 96" and name == "BR-96":
        return True
    elif tname == "Vishu" and name == "BR-97":  # Note: This is the same as the first one
        return True
    return False

match_name("AK", "SS")

False

In [130]:
match_name("AK", "AK- 245")


True

In [131]:
match_name("WW", "W- 298")

True

In [132]:
match_name( "SS- 123","ff",)

False

In [133]:
for index, trow in tally_data.iterrows():
    tnet = trow['Value']
    tqty = trow['Quantity']
    tname = trow['Particulars']
    tdate = trow['Date']

    if tname == 'Dlo':
        # Use loc to update the original DataFrame
        tally_data.loc[index, 'checked'] = True
        tally_data.loc[index, 'correct'] = True
        continue

    gst_date_df = gst_data[gst_data['Date'] == tdate]
    
    for gdx, row in gst_date_df.iterrows():
        if tally_data.loc[index, 'correct'] == True:
            continue
        if row['correct'] == True:
            continue

        date = row['Date']
        net = row['Net']
        qty = row['Total Tickets']
        name = row['Draw No.']
        remark = ""
        # Count the number of correct conditions
        correct_conditions = 0

        # Check the conditions
        if match_name(tname, name):
            correct_conditions += 1
        else:
            remark = "lottery type"
        if tqty == qty:
            correct_conditions += 1
        else:
            remark = "Quantity"
        if tnet == net:
            correct_conditions += 1
        else:
            remark = "Amount"

        # Update the flags based on the conditions
        if correct_conditions == 3:
            tally_data.loc[index, 'checked'] = True
            tally_data.loc[index, 'correct'] = True
            gst_data.loc[gdx, 'correct'] = True
            gst_data.loc[gdx, 'checked'] = True
            
            tally_data.loc[index, 'remark'] = None
        elif correct_conditions == 2:
            tally_data.loc[index, 'remark'] = remark
            gst_data.loc[gdx, 'checked'] = True
            

        # Continue if all three conditions are met
        if correct_conditions == 3:
            continue
            
    tally_data.loc[index, 'checked'] = True

print("complete")


complete


for index, trow in tally_data.iterrows():
    tnet = trow['Value']
    tqty = trow['Quantity']
    tname = trow['Particulars']
    tdate = trow['Date']
    if tname == 'Dlo':
        trow['checked'] = True
        trow['correct'] = True
        continue
    
    gst_date_df = gst_data[gst_data['Date'] == tdate]
    for _, row in gst_date_df.iterrows():
        if trow['correct'] == True:
            continue
        if row['correct'] == True:
            continue
        date = row['Date']
        net = row['Net']
        qty = row['Total Tickets']
        name = row['Draw No.']
        remark = ""
        # Count the number of correct conditions
        correct_conditions = 0

        # Check the conditions
        if match_name(tname, name):
            correct_conditions += 1
        else:
            remark = "lottery type"
        if tqty == qty:
            correct_conditions += 1
        else:
            remark = "Quantity"
        if tnet == net:
            correct_conditions += 1
        else:
            remark = "Amount"

        # Update the flags based on the conditions
        if correct_conditions == 3:
            trow['checked'] = True
            # row['checked'] = True
            trow['correct'] = True
            row['correct'] = True
            trow['remark'] = None
        elif correct_conditions == 2:
            trow['remark'] = remark

        # Continue if all three conditions are met
        if correct_conditions == 3:
            continue
    trow['checked'] = True
print("complete")
            

In [134]:
match_name("WW", "W- 713")

True

In [135]:
match_name('WW', 'W- 713')

True

In [138]:
gst_data

,Sl.No,Date,Draw No.,Total Tickets,TCS,Net Paid,Net,checked,correct
0,1.0,2023-04-01,W- 713,3300.0,0.0,106125.\n00,106125.0,True,True
1,2.0,2023-04-01,SS- 359,3600.0,0.0,116157.\n00,116157.0,True,True
2,3.0,2023-04-01,KN- 464,600.0,0.0,19500.0\n0,19500.0,True,True
3,4.0,2023-04-01,FF- 44,600.0,0.0,24843.0\n0,24843.0,True,True
4,5.0,2023-04-01,AK- 593,600.0,0.0,19360.0\n0,19360.0,True,True
...,...,...,...,...,...,...,...,...,...
1282,1283.0,2024-03-29,SS- 409,600.0,24.0,19524.0\n0,19524.0,True,True
1283,1284.0,2024-03-30,AK- 645,600.0,24.0,19384.0\n0,19384.0,True,False
1284,1285.0,2024-03-30,SS- 409,1200.0,48.0,38486.0\n0,38486.0,True,False
1285,Total,NaT,None,1637585.0,64493.0,56188224\n4,56188224.0,False,False


In [143]:
duplicates = tally_data[duplicate_mask]

In [144]:
with pd.ExcelWriter('output.xlsx') as writer:
    # Write the first DataFrame to a sheet named 'Tally Data'
    tally_data.to_excel(writer, sheet_name='Tally Data', index=False)
    
    # Write the second DataFrame to a sheet named 'GST Data'
    gst_data.to_excel(writer, sheet_name='Dlo Data', index=False)
    
    duplicates.to_excel(writer, sheet_name="Dupplicates", index=False)

print("DataFrames saved to Excel file successfully!")

DataFrames saved to Excel file successfully!


In [140]:
duplicate_mask = tally_data.duplicated(subset=['Date', 'Particulars'], keep=False)
duplicate_mask

0       False
1       False
2       False
3       False
4       False
        ...  
1577    False
1578    False
1579    False
1580    False
1581    False
Length: 1582, dtype: bool

In [136]:
tally_data

,Date,Particulars,Voucher Type,Narration,Quantity,Value,Gross Total,Purchase of Lottery,checked,correct,remark
0,2023-04-01,Dlo,Purchase,None,8700.0,285985.0,285985.0,285985.0,True,True,None
1,2023-04-01,WW,None,None,3300.0,106125.0,NaN,NaN,True,True,None
2,2023-04-01,SS,None,None,3600.0,116157.0,NaN,NaN,True,True,None
3,2023-04-01,KN,None,None,600.0,19500.0,NaN,NaN,True,True,None
4,2023-04-01,FF,None,None,600.0,24843.0,NaN,NaN,True,True,None
...,...,...,...,...,...,...,...,...,...,...,...
1577,2024-03-29,SS,None,None,600.0,19524.0,NaN,NaN,True,True,None
1578,2024-03-30,Dlo,Purchase,None,1800.0,57870.0,57870.0,57870.0,True,True,None
1579,2024-03-30,AK,None,None,600.0,19386.0,NaN,NaN,True,False,Amount
1580,2024-03-30,SS,None,None,1200.0,38484.0,NaN,NaN,True,False,Amount


In [137]:
import xlwings as xw
import pandas as pd

def cross_check_by_trader(gst_file, tally_file, gst_sheet_name='Sheet1', tally_sheet_name='Sheet1'):
    try:
        # Open the workbooks
        gst_wb = xw.Book(gst_file)
        tally_wb = xw.Book(tally_file)

        gst_sheet = gst_wb.sheets[gst_sheet_name]
        tally_sheet = tally_wb.sheets[tally_sheet_name]

        # Load the data from the Excel sheets into DataFrames
        gst_data = pd.DataFrame(gst_sheet.range("A6").options(pd.DataFrame, index=False, expand='table').value)
        tally_data = pd.DataFrame(tally_sheet.range("A7").options(pd.DataFrame, index=False, expand='table').value)
        # pri
        # Log file for discrepancies
        with open("discrepancy_log.txt", "w") as log_file:
            log_file.write("Discrepancy Report\n")
            log_file.write("="*50 + "\n")
            
            for date in gst_data

            # Loop through each trader in GST data
            for trader in gst_data['name'].unique():
                trader_gst = gst_data[gst_data['name'] == trader]
                if trader in tally_data['name'].unique():
                    trader_tally = tally_data[tally_data['name'] == trader]

                    for i, gst_row in trader_gst.iterrows():
                        # Find matching rows in tally data by Invoice Number
                        match = trader_tally[trader_tally['amount'] == gst_row['amount']]

                        if match.empty:
                            # Log missing entry in Tally for this GST invoice
                            log_file.write(f"Missing entry in Tally for GST Invoice: {gst_row['amount']} (Trader: {trader})\n")
                            gst_sheet.range(f"A{i+2}:Z{i+2}").color = (255, 0, 0)  # Red background for missing in Tally
                        else:
                            for col in ['amount', 'date']:  # Add other columns as needed
                                if gst_row[col] != match.iloc[0][col]:
                                    log_file.write(f"Discrepancy for Trader: {trader}, Invoice {gst_row['amount']} in {col}: "
                                                   f"GST: {gst_row[col]} vs Tally: {match.iloc[0][col]}\n")
                                    # Highlight the entire row red in both files
                                    gst_sheet.range(f"A{i+2}:Z{i+2}").api.Font.Color = 255  # Red font for GST file
                                    tally_sheet.range(f"A{match.index[0]+2}:Z{match.index[0]+2}").api.Font.Color = 255  # Red font for Tally file
                                else:
                                    gst_sheet.range(f"A{i+2}:Z{i+2}").color = (255, 255, 0)  # Yellow for correct

                else:
                    # If trader is missing entirely in the Tally data, log and highlight rows in GST file
                    log_file.write(f"Trader '{trader}' found in GST but missing in Tally.\n")
                    for i, gst_row in trader_gst.iterrows():
                        gst_sheet.range(f"A{i+2}:Z{i+2}").color = (255, 0, 0)  # Red background

            log_file.write("="*50 + "\n")
            log_file.write("End of Report\n")

        # Save and close workbooks
        gst_wb.save()
        tally_wb.save()
        gst_wb.close()
        tally_wb.close()

        print("Discrepancy log saved to discrepancy_log.txt")

    except Exception as e:
        print(f"An error occurred: {e}")


SyntaxError: expected ':' (252095002.py, line 22)

In [ ]:

# Call the function with your file paths
cross_check_by_trader(r"C:\Users\s0656\OneDrive\Documents\Book2.xlsx", r"C:\Users\s0656\OneDrive\Documents\Book3.xlsx")


Discrepancy log saved to discrepancy_log.txt
